## Goals and Notes

[ ] Get it working - Write a book and export as epub 
- recreate [shumer novel](https://github.com/mshumer/gpt-author/blob/main/Claude_Author.ipynb) in pydantic and GPT 4



In [1]:
# imports

import time
import re
import os
# from ebooklib import epub
from ebooklib.epub import EpubBook, EpubHtml, EpubItem, EpubNcx, EpubNav, write_epub
import base64
import requests
import json

import os
import enum
import instructor
from instructor import llm_validator
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime
from typing import Tuple, Optional, List, Annotated, ClassVar, Union
from pydantic import BaseModel, PositiveInt, Field, ValidationError, BeforeValidator, field_validator, conlist, ConfigDict, constr
from io import BytesIO
from PIL import Image
import base64

In [2]:
# load API key

dotenv_path = Path(r"C:\Storage\python_projects\ashvin\.env")
image_folderpath = Path(r"C:\Storage\python_projects\ashvin\sandbox\pydantic")
load_dotenv(dotenv_path=dotenv_path)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# main constants

GPT_MODEL_TEXT_ALIAS = "gpt-4-turbo-preview" # points to latest GPT model
GPT_MODEL_TEXT = "gpt-4-0125-preview"
GPT_MODEL_35_TEXT_ALIAS = "gpt-3.5-turbo" # points to latest GPT 3.5 Turbo model
DALL_E_3 = "dall-e-3"

#instantiate client
client = instructor.patch(OpenAI())

In [10]:
# config

min_length = 33
max_retries = 5
model=GPT_MODEL_TEXT_ALIAS
response_model = None
context = None

num_chapters = 5
chapter_length = 3
author = "sarantium"
bookid = "id12345"
cover_image_path = image_folderpath / "cover.png"
writing_style = """
A snarky comedy with a gripping sci fi or fantasy storyline. Always show not tell. Dialogue over exposition. Use active voice.
"""
topic = "a shitty superpower - can transform into a giant panda and heals instantly between shifting forms. A focus on the details of this power's usage by the superhero or supervillain."



In [ ]:
# editing - remove first line

def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

In [4]:
# my wrapper 

def wrapper(prompt: str, data: str | list, response_model: BaseModel | None = None):
    """Wrapper function to generate LLM completion"""
    response = client.chat.completions.create(
        model=GPT_MODEL_TEXT_ALIAS,
        response_model=response_model,
        max_retries=max_retries,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": data},
        ]
    )
    # Assuming response.choices[0].message.content for OpenAI response structure
    response_text = response.choices[0].message.content
    return response_text.strip()

# image wrapper function

def image_wrapper(image_prompt: str) -> str:
    """Generate an image using DALL-E 3 and return the base64 JSON representation."""
    response = client.images.generate(
        model=DALL_E_3,
        prompt=image_prompt,
        size="1024x1024",
        quality="standard",
        style="vivid",
        response_format="b64_json",
        n=1,
    )
    return response.data[0].b64_json

# save image function

def save_image(b64_json: str, folder_path: Union[str, Path], image_name: str) -> None:
    """Convert a base64 JSON image string to a PNG file and save it in the specified folder with the given image name."""
    if not isinstance(folder_path, Path):
        folder_path = Path(folder_path)
    
    # Combine the folder path and image name to create the full file path
    file_path = folder_path / f"{image_name}.png"
    
    image_data = base64.b64decode(b64_json)
    image = Image.open(BytesIO(image_data))
    image.save(file_path)

In [11]:
# prompts

plot_prompt = "You are a world-class short story author. Write the requested content with great skill and attention to detail."
plot_data = f""" 
Write me a plot outline for a {num_chapters}-chapter short story in the {writing_style} style,
based on the following topic: {topic}. Each chapter should be {chapter_length} pages long.
Only return the plot outline. No title, preamble or postscript.
"""

In [12]:
# test

plot_outline = wrapper(plot_prompt, plot_data)
print(plot_outline)

**Chapter 1: Unbearable Beginnings**

- Setting: Modern-day bustling city, vibrant and chaotic, teeming with both ordinary and superhuman activities.

- Introduction of the protagonist, Alex, a dry-humored individual with the unfortunate superpower of transforming into a giant panda. While in panda form, Alex heals instantly from any injuries but cannot communicate or control the transformation timing well.

- Scene: A casual dinner with friends at a local diner. Mid-bite into a spicy chicken wing, Alex sneezes and transforms, causing pandemonium. The diner's patrons react in a mix of horror, amusement, and curiosity. Alex, now a giant panda, clumsily tries to exit without causing more havoc, demonstrating the first glimpse of instant healing as a wobbly table leg breaks and pierces through, only for the wound to close seamlessly as Alex shifts back.

- Dialogue focuses on friends teasing Alex about the "unbearable" superpower, highlighting the snarky tone and setting the stage for bot

In [ ]:
# # prompts

# story_prompt = "You are a world-class short story author. Write the requested content with great skill and attention to detail."
# story_data = "write me a short story about a shitty superpower. Only return the story. No title, preamble or postscript."

# # test

# story = wrapper(story_prompt, story_data)
# print(story)

In [13]:
# cover prompt

cover_prompt = """
You are a world-class illustrator and cover designer for books. Write the requested content with great skill and attention to detail.
Describe the cover we should create, based on the plot. This should be two sentences long, maximum.
"""

def generate_cover_prompt(prompt, plot):
    response = wrapper(prompt, plot)
    return response

In [14]:
# test 

cover_description = generate_cover_prompt(cover_prompt, plot_outline)
print(cover_description)

The cover should feature a giant panda in a bustling cityscape, with elements of chaos and humor - like a broken diner sign and a scattering of superheroes in the background. Alex, the panda, is caught mid-sneeze, transforming back to human with a bewildered expression and a spicy chicken wing flying in the air.


In [15]:
# generate cover image

cover_image_prompt = """
You are a world-class illustrator and cover designer for visual books. Design the requested content with great skill and attention to detail.
Design the cover we should create, based on the cover description.
This is a visual book with no text or description in the cover.
"""

def create_cover_image(prompt, summary):
    full_cover_image_prompt = f"Cover image prompt: {prompt}, Summary: {summary}"
    response = image_wrapper(full_cover_image_prompt)
    return response

In [16]:
# test cover image

cover_image_b64 = create_cover_image(cover_image_prompt, cover_description)
save_image(cover_image_b64, image_folderpath, "cover")

In [17]:
# title prompt

title_prompt = """
You are a world-class publisher. Write the requested content with great skill and attention to detail.
Respond with a great title for this short story, based on the plot. Only respond with the title, nothing else is allowed.
"""

def generate_title(prompt, plot):
    response = wrapper(prompt, plot)
    return response

In [18]:
# test

title = generate_title(title_prompt, plot_outline)
print(title)

"Pandamonium: A Superhero's Unlikely Tale"


In [19]:
# generate chapter title

chapter_title_prompt = """
You are a world-class publisher. Write the requested content with great skill and attention to detail.
Respond with a great title for this chapter, based on the plot. Only respond with the title, nothing else is allowed.
No quotation marks.
"""

def generate_chapter_title(prompt, chapter_content):
    response = wrapper(prompt, chapter_content)
    return response

In [20]:
# my version of generate book

def generate_book(plot_outline, num_chapters, chapter_length, writing_style):
    chapters = []
    for i in range(1, num_chapters + 1):
        print(f"Generating chapter {i}...")
        previous_chapters = ' '.join(chapters)  # Gather all previous chapters into a single string
        
        # Prepare the prompt and data strings
        prompt = f"""
        Writing Style: {writing_style}
        Plot Outline: {plot_outline}
        Task: 
        Write chapter {i}, ensuring it follows the plot outline and builds on the previous chapters. 
        The chapter is {chapter_length} pages long.
        Special Instructions : Remember, each chapter and the overall book is a connected piece of writing. The only markdown structure in a chapter is its title.
        """
        data = f"Previous Chapters: {previous_chapters}"
        
        # Call the wrapper function with prompt and data
        chapter = wrapper(prompt, data)
        chapters.append(chapter)
        print(f"Chapter {i} complete")
        time.sleep(1)  # Delay to manage API call rates, if applicable

    print("Compiling the book...")
    book = "\n\n".join(chapters)
    print("Book generated!")

    return book, chapters



In [21]:
# test

book, chapters = generate_book(plot_outline, num_chapters, chapter_length, writing_style)

Generating chapter 1...
Chapter 1 complete
Generating chapter 2...
Chapter 2 complete
Generating chapter 3...
Chapter 3 complete
Generating chapter 4...
Chapter 4 complete
Generating chapter 5...
Chapter 5 complete
Compiling the book...
Book generated!


In [22]:
# create epub

def create_epub(id, title, author, chapters, cover_image_path, epub_save_path):
    # Function to sanitize file names
    def sanitize_filename(filename):
        """Remove invalid characters from filenames."""
        import re
        return re.sub(r'[\\/*?:"<>|]', "", filename).strip()

    book = EpubBook()
    # Set metadata
    book.set_identifier(id)
    book.set_title(title)
    book.set_language('en')
    book.add_author(author)
    # Add cover image
    with open(cover_image_path, 'rb') as cover_file:
        cover_image = cover_file.read()
    book.set_cover('cover.png', cover_image)
    # Create chapters and add them to the book
    epub_chapters = []
    for i, chapter_content in enumerate(chapters, start=1):
        chapter_title = generate_chapter_title(chapter_title_prompt, chapter_content)  # ensure generate_chapter_title is defined
        chapter_file_name = f'chapter_{i}.xhtml'
        epub_chapter = EpubHtml(title=chapter_title, file_name=chapter_file_name, lang='en')
        # Add paragraph breaks and ensure HTML line breaks
        formatted_content = ''.join(f'<p>{paragraph.strip()}</p>' for paragraph in chapter_content.split('\n') if paragraph.strip())
        # Include two HTML <br> tags for line breaks
        epub_chapter.content = f'<h1>Chapter {i}: {chapter_title}</h1><br><br>{formatted_content}'
        book.add_item(epub_chapter)
        epub_chapters.append(epub_chapter)
        time.sleep(1)

    # Define Table of Contents
    book.toc = (epub_chapters)

    # Add default NCX and Nav files
    book.add_item(EpubNcx())
    book.add_item(EpubNav())

    # Define CSS style
    style = '''
    @namespace epub "http://www.idpf.org/2007/ops";
    body {
        font-family: Cambria, Liberation Serif, serif;
    }
    h1 {
        text-align: left;
        text-transform: uppercase;
        font-weight: 200;
    }
    p {
        margin: 0;
        padding: 0;
        text-align: justify;
    }
    '''

    # Add CSS file
    nav_css = EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)
    book.add_item(nav_css)

    # Create spine
    book.spine = ['nav'] + epub_chapters

    # Save the EPUB file
    sanitized_title = sanitize_filename(title)
    save_path = epub_save_path / f'{sanitized_title}.epub'
    print(f"Saving EPUB at: {save_path}")
    write_epub(save_path, book)


In [23]:
# test

create_epub(id=bookid, title=title, author=author, chapters=chapters, cover_image_path=cover_image_path, epub_save_path=image_folderpath)

Saving EPUB at: C:\Storage\python_projects\ashvin\sandbox\pydantic\Pandamonium A Superhero's Unlikely Tale.epub
